# 売上資料Ⅳ-(2)
## 月別Class別大分類別粗利　作成

- 作成年度と作成月の入力を忘れずに！
- ICAST売上資料Ⅷの作成データを使用する。

In [ ]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def data_read():
    # 売上
    df_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/売上資料Ⅷ.csv",  encoding='cp932')
    
    return df_i

In [ ]:
# 作成年度を入力
nendo = 2021
# 作成月を入力（1～12を入力）
month = 5

# データ読込
df_i = data_read()

In [ ]:
def month_set(nendo, month):
    """年と月で6桁の年月にする
    """
    if month < 6:
        this_month = int(str(nendo+1) + "0" + str(month))
    else:
        if month > 9:
            this_month = int(str(nendo) + str(month))
        else:
            this_month = int(str(nendo) + "0" + str(month))
    
    return this_month

In [ ]:
# 当月を設定
this_month = month_set(nendo, month)

# 当年の最終月を設定
end_month = month_set(nendo, 5)

# 12ヵ月分のlistを作成
list_12 = []
m = 6
y = month_set(nendo, m)
list_12.append(y)
for i in range(11):
    m += 1
    if m > 12:
        m = 1
    y = month_set(nendo, m)
    list_12.append(y)
print('当月：', this_month)
print('最終月', end_month)
print(list_12)

当月： 202205
最終月 202205
[202106, 202107, 202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204, 202205]


### データ作成

In [ ]:
# 売上データ絞り込み
df_i = df_i[(df_i["対象月"] >= list_12[0]) & (df_i["対象月"] <= list_12[-1])]


In [ ]:
class_dai_dict = {}

for i in range(len(df_i)):
    classcd = df_i.iloc[i]['Classcd']
    dai = df_i.iloc[i]['大分類CD']
    taisyo = df_i.iloc[i]['対象月']
    if (classcd, dai, taisyo) in class_dai_dict:
        class_dai_dict[classcd, dai, taisyo][0] += df_i.iloc[i]['販売金額']
        class_dai_dict[classcd, dai, taisyo][1] += df_i.iloc[i]['期首在庫金額']
        class_dai_dict[classcd, dai, taisyo][2] += df_i.iloc[i]['仕入金額']
        class_dai_dict[classcd, dai, taisyo][3] += df_i.iloc[i]['期末在庫金額']
    else:
        class_dai_dict[classcd, dai, taisyo] = [df_i.iloc[i]['販売金額'], df_i.iloc[i]['期首在庫金額'],\
                                              df_i.iloc[i]['仕入金額'], df_i.iloc[i]['期末在庫金額'],\
                                              df_i.iloc[i]['Class'], df_i.iloc[i]['大分類']]

In [ ]:
class_dai_list = []

for i, k in class_dai_dict.items():
    class_dai_list.append([i[0],k[4],i[1],k[5],i[2],list_12.index(i[2]),0,k[0]])
    class_dai_list.append([i[0],k[4],i[1],k[5],i[2],list_12.index(i[2]),1,k[1]])
    class_dai_list.append([i[0],k[4],i[1],k[5],i[2],list_12.index(i[2]),2,k[2]])
    class_dai_list.append([i[0],k[4],i[1],k[5],i[2],list_12.index(i[2]),3,k[3]])

In [ ]:
col_nm = ['Classcd','Class名','大分類CD','大分類名','売上月','月番号','処理区分','金額']
df = pd.DataFrame(class_dai_list, columns=col_nm)

In [ ]:
df = df.sort_values(['Classcd','大分類CD','売上月'])

In [ ]:
# Excelデータ作成
df.to_excel("/content/drive/MyDrive/Purchased_products/excel/月別Class別大分類別粗利表.xlsx", sheet_name='粗利表')